# Timeseries download - work in progress

So far, this script only downloads solar and wind timeserie from several european TSOs

#####Loading some python libraries and defining downloads path inside the notebook's directory on the users computer

In [8]:
import urllib
import os
from time import sleep
from dateutil import rrule
from dateutil.relativedelta import relativedelta
from datetime import datetime

path = 'downloads/'
if not os.path.exists(path): os.makedirs(path)

#####This is a python dictionary containing input parameters needed to generate the URLs belonging to the data sources

In [13]:
conf = {
    'Fingrid': {
        'wind': {
            'url_template': 'http://www.fingrid.fi/en/electricity-market/load-and-generation/windpower/Pages/default.aspx/Excel/TimeSeries.xls?beginDate=20150101&endDate=20151231&variables=H4sIAAAAAAAEAO29B2AcSZYlJi9tynt_SvVK1-B0oQiAYBMk2JBAEOzBiM3mkuwdaUcjKasqgcplVmVdZhZAzO2dvPfee--999577733ujudTif33_8_XGZkAWz2zkrayZ4hgKrIHz9-fB8_Ih7_ZFYX2aTMmyP769nyvDp6_CJb5EfPzn7_N1999fzsJ788--L49_99vv3m8V3-_PHZ8jKv25_MynV-dJ6VTf74rv_RY_7xrKoXWfu6rYvlxdGPj378x3ce3-1_8firZdG-yd-1R198d353_viu_fvxm6It86PvFstZuqqu8jq9yJd5nbVFtUy3uPWdx3el0ePnxTI_qcqqTu96f3y7XZRHP_7k5MnDHeo8_Pjx63l19eWyvH62btd1_jRrMzOWyDdA3aeP_bM5-n8AhQmq0kUBAAA1&cultureId=en-US&dataTimePrecision=5',
            'bundle': 'YEARLY',
            'start': '2014-11-28',
            'end': '2015-01-01',
        }
    }  
    ,'Elia': {
        'wind1': {
            'url_template': 'http://publications.elia.be/Publications/Publications/WindForecasting.v2.svc/ExportForecastData?beginDate={u_start:%Y-%m-%d}T23%3A00%3A00.000Z&endDate={u_end:%Y-%m-%d}T23%3A00%3A00.000Z&isOffshore=&isEliaConnected=',
            'bundle': 'MONTHLY',
            'start': '2012-02-01',
            'end': '2015-01-01',
        }
        ,'pv': {
            'url_template': 'http://publications.elia.be/Publications/Publications/SolarForecasting.v3.svc/ExportSolarForecastGraph?dateFrom={u_start:%Y-%m-%d}T23%3A00%3A00.000Z&dateTo={u_end:%Y-%m-%d}T23%3A00%3A00.000Z&sourceId=1',
            'bundle': 'MONTHLY',
            'start': '2012-11-14',
            'end': '2015-01-01',
        }
    }  
    ,'Amprion': {
        'wind': {
            'url_template': 'http://preview.amprion.de/applications/applicationfiles/winddaten.php?mode=download&format=csv&start={u_start:%Y.%m.%d}&end={u_end:%Y.%m.%d}',
            'bundle': 'complete',
            'start': '2006-01-03',
            'end': '2015-01-01',
        }
        ,'pv': {
            'url_template': 'http://amprion.de/applications/applicationfiles/PV_einspeisung.php?mode=download&format=csv&start={u_start:%Y.%m.%d}&end={u_end:%Y.%m.%d}',
            'bundle': 'complete',
            'start': '2010-01-07',
            'end': '2015-01-01',
        }
    }    
    ,'CEPS': {
        'wind_pv': {
            'url_template': 'http://www.ceps.cz/_layouts/15/Ceps/_Pages/GraphData.aspx?mode=xlsx&from={u_start:%m.%d.%Y}%2012:00:00%20AM&to={u_end:%m/%d/%Y}%2011:59:59%20PM&hasinterval=False&sol=26&lang=ENG&agr=QH&fnc=SUM&ver=RT&para1=all&',
            'bundle': 'complete',
            'start': '2012-01-01',
            'end': '2015-01-01',
        }
    }    
    ,'TransnetBW': {
        'wind': {
            'url_template': 'https://www.transnetbw.de/de/kennzahlen/erneuerbare-energien/windenergie?app=wind&activeTab=csv&selectMonatDownload={month}&view=1&download=true',
            'bundle': 'special',
            'start': '2010-01-01',
            'end': '2015-01-01',
        }
        ,'pv': {
            'url_template': 'https://www.transnetbw.de/de/kennzahlen/erneuerbare-energien/fotovoltaik?app=solar&activeTab=csv&selectMonatDownload={month}&view=1&download=true',
            'bundle': 'special',
            'start': '2011-01-01',
            'end': '2015-01-01',
        }
    }
    ,'TenneT': {
        'wind': {
            'url_template': 'http://www.tennettso.de/site/de/phpbridge?commandpath=Tatsaechliche_und_prognostizierte_Windenergieeinspeisung%2FmonthDataSheetCsv.php&querystring=monat%3D{u_start:%Y-%m}&contenttype=text%2Fx-csv',
            'bundle': 'MONTHLY',
            'start': '2005-07-13',
            'end': '2015-01-01',
        }
        ,'pv': {
            'url_template': 'http://www.tennettso.de/site/de/phpbridge?commandpath=Tatsaechliche_und_prognostizierte_Solarenergieeinspeisung%2FmonthDataSheetCsv.php&sub=total&querystring=monat%3D{u_start:%Y-%m}&contenttype=text%2Fx-csv',
            'bundle': 'MONTHLY',
            'start': '2010-01-01',
            'end': '2015-01-01',
        }
    }
    ,'50hertz': {
        'wind': {
            'url_template': 'http://ws.50hertz.com/web01/api/WindPowerForecast/DownloadFile?fileName={u_start:%Y}.csv&callback=?',
            'bundle': 'YEARLY',
            'start': '2005-01-01',
            'end': '2015-01-01',
        }
        ,'pv': {
            'url_template': 'http://ws.50hertz.com/web01/api/PhotovoltaicForecast/DownloadFile?fileName={u_start:%Y}.csv&callback=?',
            'bundle': 'YEARLY',
            'start': '2012-01-01',
            'end': '2015-01-01',
        }
    }
    ,'RTE': {
        'wind_pv': {
            'url_template': 'http://clients.rte-france.com/servlets/RealProdServlet?annee={u_start:%Y}',
            'bundle': 'YEARLY',
            'start': '2014-01-01',
            'end': '2015-01-01', 
        }
    }
    ,'Svenska_Kraftnaet': {
        'wind_pv_1': {
            'url_template': 'http://www.svk.se/siteassets/aktorsportalen/statistik/sverigestatistik/n_fot{u_start:%Y}.xls',
            'bundle': 'YEARLY',
            'start': '2002-01-01',
            'end': '2009-01-01',
        }
        ,'wind_pv_2': {
            'url_template': 'http://www.svk.se/siteassets/aktorsportalen/statistik/sverigestatistik/n_fot201001-06.xls',
            'bundle': 'YEARLY',
            'start': '2010-01-01',
            'end': '2010-01-01',
        }
        ,'wind_pv_3': {
            'url_template': 'http://www.svk.se/siteassets/aktorsportalen/statistik/sverigestatistik/n_fot2010-07-12.xls',
            'bundle': 'YEARLY',
            'start': '2010-01-01',
            'end': '2010-01-01',
        }
        ,'wind_pv_4': {
            'url_template': 'http://www.svk.se/siteassets/aktorsportalen/statistik/sverigestatistik/n-fot2011-01-12.xls',
            'bundle': 'YEARLY',
            'start': '2011-01-01',
            'end': '2011-01-01',
        }
        ,'wind_pv_5': {
            'url_template': 'http://www.svk.se/siteassets/aktorsportalen/statistik/sverigestatistik/n_fot{u_start:%Y}-01-12.xls',
            'bundle': 'YEARLY',
            'start': '2012-01-01',
            'end': '2014-01-01',
        }
    }
    ,'OeMag': {
        'wind_1': {
            'url_template': 'http://www.oem-ag.at/fileadmin/user_upload/Dokumente/statistik/winderzeugung/winderzeugung_{u_start:%Y}.xls',
            'bundle': 'YEARLY',
            'start': '2003-01-01',
            'end': '2011-01-01',
        }
        ,'wind_2': {
            'url_template': 'http://www.oem-ag.at/fileadmin/user_upload/Dokumente/statistik/winderzeugung/winderzeugung_{u_start:%Y}.xlsx',
            'bundle': 'YEARLY',
            'start': '2012-01-01',
            'end': '2012-01-01',
        }
    }            
}

#####Here we iterate over the sources and technology (wind/solar) entries specified above and download the data for all periods (depending on whats specified under 'bundle' either years, days or months) 

In [16]:
def download1(url_template, source, tech, start, end):
    global i
    if i < 1: #for purpose of faster testing, limit number of files downloaded per technology per TSO to 1
        if source == 'Elia': #Elia still causes some problems with the url, this only partly fixes it
            start = start + relativedelta(days = -1)
            end = end + relativedelta(days = -1)
        filename = source+'_'+tech+'_'+start.strftime('%Y-%m-%d')+'_'+end.strftime('%Y-%m-%d')
        full_url = url_template.format(u_start = start, u_end = end)
        download_url(full_url, filename)
        i+=1
    else:
        return
    
def download_TransnetBW(url_template, count, source, tech):
    name = source+'_'+tech+'_'+str(count)
    full_url = url_template.format(month = count)
    download_url(full_url, name)

def download_url(full_url, filename):
    print('attempting download of ', filename)    
    print(full_url)
    sleep(1) # how can I get the funtion to first print the messages, then do the download (this doesn't work)?
    file = urllib.request.urlopen(full_url)
    data = file.read()
    with open(path+filename+'.csv', 'wb') as output_file:
        output_file.write(data)
    print('done')    
      
for source, tech in conf.items():
    for tech, param in tech.items():
        i = 0
        if source == 'TransnetBW': #TransnetBW just counts the months backwards, this requires a different approach
            for count in range(0,70): #iterates over range from 0 to 69
                download_TransnetBW(param['url_template'], count, source, tech)
            continue #the following steps apply if TSO is not TransnetBW
        start = datetime.strptime(param['start'], '%Y-%m-%d').date()
        end = datetime.strptime(param['end'], '%Y-%m-%d').date()
        period = start
        if param['bundle'] == 'complete':
            download1(param['url_template'], source, tech, start, end)
        if param['bundle'] == 'YEARLY':
            for period in rrule.rrule(rrule.YEARLY, dtstart=start, until=end):
                p_start = period.replace(month = 1, day = 1)
                p_end = p_start + relativedelta(years = 1)
                download1(param['url_template'], source, tech, p_start, p_end)
        if param['bundle'] == 'MONTHLY':
            for period in rrule.rrule(rrule.MONTHLY, dtstart=start, until=end):
                p_start = period.replace(day = 1)
                p_end = p_start + relativedelta(months = 1)
                download1(param['url_template'], source, tech, p_start, p_end)                            
        if param['bundle'] == 'DAILY':
            for period in rrule.rrule(rrule.DAILY, dtstart=start, until=end):
                download1(param['url_template'], period, source, tech)            

attempting download of  Fingrid_wind_2014-01-01_2015-01-01
http://www.fingrid.fi/en/electricity-market/load-and-generation/windpower/Pages/default.aspx/Excel/TimeSeries.xls?beginDate=20150101&endDate=20151231&variables=H4sIAAAAAAAEAO29B2AcSZYlJi9tynt_SvVK1-B0oQiAYBMk2JBAEOzBiM3mkuwdaUcjKasqgcplVmVdZhZAzO2dvPfee--999577733ujudTif33_8_XGZkAWz2zkrayZ4hgKrIHz9-fB8_Ih7_ZFYX2aTMmyP769nyvDp6_CJb5EfPzn7_N1999fzsJ788--L49_99vv3m8V3-_PHZ8jKv25_MynV-dJ6VTf74rv_RY_7xrKoXWfu6rYvlxdGPj378x3ce3-1_8firZdG-yd-1R198d353_viu_fvxm6It86PvFstZuqqu8jq9yJd5nbVFtUy3uPWdx3el0ePnxTI_qcqqTu96f3y7XZRHP_7k5MnDHeo8_Pjx63l19eWyvH62btd1_jRrMzOWyDdA3aeP_bM5-n8AhQmq0kUBAAA1&cultureId=en-US&dataTimePrecision=5
done
attempting download of  RTE_wind_pv_2014-01-01_2015-01-01
http://clients.rte-france.com/servlets/RealProdServlet?annee=2014
done
attempting download of  CEPS_wind_pv_2012-01-01_2015-01-01
http://www.ceps.cz/_layouts/15/Ceps/_Pages/GraphData.aspx?mode=xlsx&from=01.01.2012%2012:00:00%20AM&to=01/01/2015%2011:59:5